In [ ]:
#r "nuget: MathNet.Numerics"
#r "nuget: MathNet.Numerics.FSharp"

Installed Packages MathNet.Numerics, 4.15.0 MathNet.Numerics.FSharp, 4.15.0

In [ ]:
open MathNet.Numerics
open MathNet.Numerics.LinearAlgebra
open MathNet.Numerics.LinearAlgebra.Vector
open MathNet.Numerics.LinearAlgebra.Double

In [ ]:
// target function
let regressionCurve coef input = 
    coef
    |> Vector.foldi (fun i acc item -> acc + item * Math.Pow (input, double i)) 0.0

// mean squared error
// datasets [[X0, y0]; [X1, y1]; [X2, y2]]
let lsmDerivEquation coefNum dataSets =
    let dataNum = dataSets |> Matrix.rowCount
    
    // left -> X
    let eqCoefsLeft = 
        DenseMatrix.init coefNum coefNum
            (fun row col ->
                dataSets
                |> Matrix.foldRows
                    (fun acc item -> acc + Math.Pow (item.[0], row + col |> double))
                    0.0
            )
        
    // right -> y
    let eqCoefRight = 
        DenseVector.init coefNum
            (fun pos ->
                dataSets 
                |> Matrix.foldRows 
                    (fun acc item -> acc + item.[1] * Math.Pow (item.[0], pos |> double)) 
                    0.0
            )
    
    (eqCoefsLeft, eqCoefRight)

// solve regression curve's coefficients
let regressionCurveCoef degree dataSets = 
    let x, y = lsmDerivEquation (degree + 1) dataSets
    x.Solve(y)

In [ ]:
let dataSets = 
    matrix [[0.000000; 0.073346];
            [0.100000; 0.663607];
            [0.200000; 0.794265];
            [0.300000; 1.045710];
            [0.400000; 0.265533];
            [0.500000; -0.452839];
            [0.600000; -0.297819];
            [0.700000; -1.061552];
            [0.800000; -0.961003];
            [0.900000; -0.248203]]

In [ ]:
let coef = regressionCurveCoef 6 dataSets
coef

index,value
0,0.08541222409992137
1,0.3556025522846945
2,89.18244729672175
3,-532.0098740708634
4,1163.2473082114625
5,-1137.857257763437
6,420.8131249161627


In [ ]:
let regressionCurveWithCoef = regressionCurve coef

In [ ]:
#r "nuget: xplot.plotly"
#r "nuget: xplot.googlecharts"
#r "nuget: XPlot.Plotly.Interactive"

Installed Packages xplot.googlecharts, 3.0.1 xplot.plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.6

In [ ]:
open XPlot.Plotly

In [ ]:
let layout =
    Layout(
        xaxis = Xaxis(range = [-0.1; 1.1], zeroline = false),
        yaxis = Yaxis(range = [-1.1; 1.1], zeroline = false)
    )

let curveList = 
    [1..9]
    |> List.map 
        (fun item -> 
            Scatter(
                x = [for x in -1.1 .. 0.01 .. 1.1 -> x],
                y = [for x in -1.1 .. 0.01 .. 1.1 -> regressionCurve (regressionCurveCoef item dataSets) x],
                
                mode = "lines"
            )
        )

let dataPoints = 
    Scatter(
        x = dataSets.Column(0),
        y = dataSets.Column(1),
        mode = "markers"
    )

let labels = 
    seq{
        "data points"
        for i in 1..9 do
            "M: " + (string i)
    }

dataPoints :: curveList
|> Chart.Plot
|> Chart.WithLabels labels
|> Chart.WithLayout layout
|> Chart.WithHeight 500
|> Chart.WithWidth 800

<!DOCTYPE html>